# RFM + Churn prediction

- Part 1: Import & Load Data
- Part 2: Data Quality Assessment
- Part 3: Data Cleaning
- Part 4: Feature Engineering
- Part 5: EDA
- Part 6: RFM Analysis
- Part 7: Customer Segmentation & Churn Prediction
  - 7.1 Data Preparation
  - 7.2 K Selection
  - 7.3 Normalization & Algorithm Comparison
  - 7.4 Final Clustering & Pareto Analysis
  - 7.5 Churn Definition
  - 7.6 Time-based Split
  - 7.7 RFM Features & Churn Label
  - 7.8 Churn Prediction Model
  - 7.9 Model Comparison
  - 7.10 Cross Validation
  - 7.11 Confusion Matrix
  - 7.12 ROC Curve
  - 7.13 Feature Importance
- Part 8: Model Deployment (Gradio)
- Part 9: Conclusion & Recommendations

## Part 1. Import Libraries and Load Data
### 1.1 Import Libraries

In [954]:
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import joblib
import os
import gradio as gr
import numpy as np

In [955]:
# Load the dataset
data = pd.read_excel('../data/Online Retail.xlsx')

In [956]:
data.head() 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Part 2. Data Quality Assessment

In [957]:
# 2.1 Check data structure
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [958]:
# 2.2 Summary statistics
data.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [959]:
# 2.3 Check missing values
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [960]:
# Check anomalies
print(f"Negative Quantity: {(data['Quantity'] < 0).sum():,}")
print(f"Zero/Negative UnitPrice: {(data['UnitPrice'] <= 0).sum():,}")
print(f"Cancelled Orders: {data['InvoiceNo'].astype(str).str.startswith('C').sum():,}")

Negative Quantity: 10,624
Zero/Negative UnitPrice: 2,517
Cancelled Orders: 9,288


**Findings:** 135,080 missing CustomerID and description 1454 , 10,624 negative quantities (returns), 2,517 invalid prices.

- drop missing value CustID and Ignore Description

## Part 3. Data Cleaning

In [961]:
# Convert datetime & create TotalAmount
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [962]:
data.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [963]:
# Before cleaning
print(f"Before: {len(data):,} rows")

Before: 541,909 rows


In [964]:
# Remove rows with missing CustomerID
data_clean = data.dropna(subset=['CustomerID'])
print(f"After removing missing CustomerID: {len(data_clean):,} rows")

After removing missing CustomerID: 406,829 rows


In [965]:
# Remove rows with negative Quantity
data_clean = data_clean[data_clean['Quantity'] > 0]
print(f"After removing invalid Quantity: {len(data_clean):,} rows")

After removing invalid Quantity: 397,924 rows


In [966]:
data_clean = data_clean[data_clean['UnitPrice'] > 0]
print(f"After removing invalid UnitPrice: {len(data_clean):,} rows")

After removing invalid UnitPrice: 397,884 rows


In [967]:
# Customer count after cleaning
print(f"\n=== Customer Count ===")
print(f"Unique customers: {data_clean['CustomerID'].nunique():,}")


=== Customer Count ===
Unique customers: 4,338


In [968]:
# Verify cleaning
print(f"\n=== Cleaning Summary ===")
print(f"Original: 541,909 rows")
print(f"Final: {len(data_clean):,} rows")
print(f"Removed: {541909 - len(data_clean):,} rows ({(541909 - len(data_clean))/541909*100:.1f}%)")


=== Cleaning Summary ===
Original: 541,909 rows
Final: 397,884 rows
Removed: 144,025 rows (26.6%)


**Result:** Cleaned dataset contains 397,884 valid transactions (removed 26.6%).

## Part 4. Create features for EDA

In [969]:
# Create TotalAmount column for Monetary analysis
data_clean['TotalAmount'] = data_clean['Quantity'] * data_clean['UnitPrice']
# Time features
data_clean['Hour'] = data_clean['InvoiceDate'].dt.hour
data_clean['DayOfWeek'] = data_clean['InvoiceDate'].dt.day_name()
data_clean['Month'] = data_clean['InvoiceDate'].dt.month
data_clean['YearMonth'] = data_clean['InvoiceDate'].dt.to_period('M').astype(str)

print(f"Features created: Revenue, Hour, DayOfWeek, Month, YearMonth")
print(f"Data shape: {data_clean.shape}")


Features created: Revenue, Hour, DayOfWeek, Month, YearMonth
Data shape: (397884, 13)


In [970]:
data_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount,Hour,DayOfWeek,Month,YearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,8,Wednesday,12,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,8,Wednesday,12,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,8,Wednesday,12,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,8,Wednesday,12,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,8,Wednesday,12,2010-12


## Part 5. Exploratory Data Analysis (EDA)
- 5.1 Time Analysis
    - Day of Week 
    - Saturday check
    - Transactions by Hour
    - Peak Hour
    - Basket Size

### 5.1.1 Day of Week Analysis

In [971]:
day_count = data_clean.groupby('DayOfWeek')['InvoiceNo'].nunique()
day_count = day_count.reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

fig = px.bar(x=day_count.index, y=day_count.values,
             title='Orders by Day of Week',
             labels={'x': 'Day', 'y': 'Number of Orders'},
             color=day_count.values, color_continuous_scale='Viridis')
fig.update_layout(showlegend=False)
fig.show()

print(f"Busiest Day: {day_count.idxmax()} ({day_count.max():,} orders)")
print(f"Saturday transactions: {day_count.get('Saturday', 0)} - Business closed on Saturdays")

Busiest Day: Thursday (4,032.0 orders)
Saturday transactions: nan - Business closed on Saturdays


In [972]:
# Check all days
data_clean['DayOfWeek'].value_counts()

DayOfWeek
Thursday     80035
Wednesday    68885
Tuesday      66473
Monday       64893
Sunday       62773
Friday       54825
Name: count, dtype: int64

In [973]:
# Check Saturday transactions
saturday_count = data_clean[data_clean['DayOfWeek'] == 'Saturday'].shape[0]
print(f"Saturday transactions: {saturday_count}")

Saturday transactions: 0


### 5.1.2 Hourly Analysis

In [974]:
hour_count = data_clean.groupby('Hour')['InvoiceNo'].nunique()
hour_count = hour_count.reindex(range(24), fill_value=0)

fig = px.bar(x=hour_count.index, y=hour_count.values,
             title='Orders by Hour',
             labels={'x': 'Hour', 'y': 'Number of Orders'},
             color=hour_count.values, color_continuous_scale='Viridis')
fig.update_layout(showlegend=False)
fig.show()

print(f"Peak Hour: {hour_count.idxmax()}:00 ({hour_count.max():,} orders)")
print(f"Shop Hours: ~7:00 - 20:00")

Peak Hour: 12:00 (3,130 orders)
Shop Hours: ~7:00 - 20:00


In [975]:
# Check transactions per hour
print(hour_count.sort_values())

Hour
0        0
21       0
22       0
5        0
23       0
3        0
2        0
1        0
4        0
6        1
20      18
7       29
19     144
18     169
17     544
8      555
16    1100
9     1393
15    2037
10    2226
14    2274
11    2277
13    2636
12    3130
Name: InvoiceNo, dtype: int64


-  Shop Open ~7:00-20:00 and Peak Hour = 12:00 

### 5.1.3 Basket Size 

In [976]:
basket_size = data_clean.groupby('InvoiceNo')['Quantity'].sum()
basket_filtered = basket_size[basket_size < 500]  # Filter outliers

fig = px.histogram(basket_filtered, title='Distribution of Basket Size',
                   labels={'value': 'Items per Transaction'},
                   nbins=50)
fig.show()

print(f"Average Basket Size: {basket_size.mean():.1f} items")
print(f"Median Basket Size: {basket_size.median():.1f} items")

Average Basket Size: 278.9 items
Median Basket Size: 155.0 items


- Note: Mean > Median indicates right-skewed distribution (some large orders)

### 5.2 Product Analysis

In [977]:
# Top 10 Products by Quantity
top_qty = data_clean.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

fig = px.bar(y=top_qty.index, x=top_qty.values, orientation='h',
             title='Top 10 Products by Quantity Sold',
             labels={'x': 'Quantity', 'y': 'Product'},
             color=top_qty.values, color_continuous_scale='Viridis')
fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.show()

# Top 10 Products by Revenue
top_rev = data_clean.groupby('Description')['TotalAmount'].sum().sort_values(ascending=False).head(10)

fig = px.bar(y=top_rev.index, x=top_rev.values, orientation='h',
             title='Top 10 Products by Revenue',
             labels={'x': 'Revenue (£)', 'y': 'Product'},
             color=top_rev.values, color_continuous_scale='Viridis')
fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.show()

print(f"Total Products: {data_clean['Description'].nunique():,}")
print(f"Top Product (Qty): {top_qty.index[0]}")
print(f"Top Product (Revenue): {top_rev.index[0]}")

Total Products: 3,877
Top Product (Qty): PAPER CRAFT , LITTLE BIRDIE
Top Product (Revenue): PAPER CRAFT , LITTLE BIRDIE


### 5.3 Country Analysis

In [978]:
# Revenue by Country (Top 10)
country_rev = data_clean.groupby('Country')['TotalAmount'].sum().sort_values(ascending=False).head(10)

fig = px.bar(x=country_rev.index, y=country_rev.values,
             title='Top 10 Countries by Revenue',
             labels={'x': 'Country', 'y': 'Revenue (£)'},
             color=country_rev.values, color_continuous_scale='Viridis')
fig.update_layout(showlegend=False)
fig.show()

# Customer Distribution - UK vs Non-UK (Pie)
uk_cust = data_clean[data_clean['Country'] == 'United Kingdom']['CustomerID'].nunique()
non_uk_cust = data_clean[data_clean['Country'] != 'United Kingdom']['CustomerID'].nunique()

fig = px.pie(values=[uk_cust, non_uk_cust], names=['UK', 'Non-UK'],
             title='Customer Distribution: UK vs Non-UK', hole=0.3,
             color_discrete_sequence=['steelblue', 'coral'])
fig.show()

# Top 10 Non-UK Countries (Bar)
non_uk_cust_detail = data_clean[data_clean['Country'] != 'United Kingdom'].groupby('Country')['CustomerID'].nunique().sort_values(ascending=False).head(10)

fig = px.bar(x=non_uk_cust_detail.index, y=non_uk_cust_detail.values,
             title='Top 10 Non-UK Countries by Customers',
             labels={'x': 'Country', 'y': 'Customers'},
             color=non_uk_cust_detail.values, color_continuous_scale='Viridis')
fig.update_layout(showlegend=False)
fig.show()

# UK vs Non-UK Summary
uk_rev = data_clean[data_clean['Country'] == 'United Kingdom']['TotalAmount'].sum()
total_rev = data_clean['TotalAmount'].sum()
print(f"UK Revenue: £{uk_rev:,.0f} ({uk_rev/total_rev*100:.1f}%)")
print(f"Non-UK Revenue: £{total_rev - uk_rev:,.0f} ({(total_rev-uk_rev)/total_rev*100:.1f}%)")
print(f"Total Countries: {data_clean['Country'].nunique()}")

UK Revenue: £7,308,392 (82.0%)
Non-UK Revenue: £1,603,016 (18.0%)
Total Countries: 37


### 5.4 Monthly Trend Analysis

In [979]:
# Monthly Revenue & Orders Trend
monthly_stats = data_clean.groupby('YearMonth').agg(
    Revenue=('TotalAmount', 'sum'),
    Orders=('InvoiceNo', 'nunique'),
    Customers=('CustomerID', 'nunique')
).reset_index()

# Create dual-axis chart
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Revenue bars
fig.add_trace(
    go.Bar(x=monthly_stats['YearMonth'].astype(str), y=monthly_stats['Revenue'],
           name='Revenue (£)', marker_color='steelblue', opacity=0.7),
    secondary_y=False
)

# Orders line
fig.add_trace(
    go.Scatter(x=monthly_stats['YearMonth'].astype(str), y=monthly_stats['Orders'],
               name='Orders', line=dict(color='coral', width=3), mode='lines+markers'),
    secondary_y=True
)

fig.update_layout(
    title='Monthly Revenue & Orders Trend',
    xaxis_title='Month',
    hovermode='x unified'
)
fig.update_yaxes(title_text='Revenue (£)', secondary_y=False)
fig.update_yaxes(title_text='Orders', secondary_y=True)
fig.show()

# MoM Growth Rate
monthly_stats['Revenue_Growth'] = monthly_stats['Revenue'].pct_change() * 100

fig2 = px.bar(monthly_stats[1:], x='YearMonth', y='Revenue_Growth',
              title='Month-over-Month Revenue Growth (%)',
              color='Revenue_Growth',
              color_continuous_scale='RdYlGn',
              color_continuous_midpoint=0)
fig2.add_hline(y=0, line_dash="dash", line_color="black")
fig2.update_layout(xaxis_title='Month', yaxis_title='Growth Rate (%)')
fig2.show()

### 5.5 Average Order Value (AOV) Analysis

In [1030]:
# Calculate AOV per order
order_aov = data_clean.groupby('InvoiceNo')['TotalAmount'].sum().reset_index()
order_aov.columns = ['InvoiceNo', 'OrderValue']

# AOV Statistics
print("=== AOV Statistics ===")
print(f"Mean AOV: £{order_aov['OrderValue'].mean():.2f}")
print(f"Median AOV: £{order_aov['OrderValue'].median():.2f}")

# AOV Distribution (filter outliers for viz)
aov_filtered = order_aov[order_aov['OrderValue'].between(0, order_aov['OrderValue'].quantile(0.95))]

fig = px.histogram(aov_filtered, x='OrderValue', nbins=50,
                   title='Order Value Distribution (excluding top 5% outliers)',
                   labels={'OrderValue': 'Order Value (£)'})
fig.add_vline(x=order_aov['OrderValue'].mean(), line_dash="dash", line_color="red",
              annotation_text=f"Mean: £{order_aov['OrderValue'].mean():.0f}")
fig.add_vline(x=order_aov['OrderValue'].median(), line_dash="dash", line_color="green",
              annotation_text=f"Median: £{order_aov['OrderValue'].median():.0f}")
fig.show()

=== AOV Statistics ===
Mean AOV: £480.87
Median AOV: £303.04


**Insight:**
- **Mean AOV (£481) > Median (£303)**: Right-skewed distribution
- **Most orders**: £200-400 range
- **Action**: Bundle deals or free shipping above £400 to increase AOV

## Part 6. RFM Analysis

In [980]:
# Set analysis date
analysis_date = data_clean['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Analysis Date: {analysis_date}")

Analysis Date: 2011-12-10 12:50:00


In [981]:
# Calculate RFM
recency = data_clean.groupby('CustomerID')['InvoiceDate'].max()
recency = (analysis_date - recency).dt.days

frequency = data_clean.groupby('CustomerID')['InvoiceNo'].nunique()

monetary = data_clean.groupby('CustomerID')['TotalAmount'].sum()

# Create RFM DataFrame
rfm = pd.DataFrame({
    'Recency': recency,
    'Frequency': frequency,
    'Monetary': monetary
})

rfm.reset_index(inplace=True)
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


In [982]:
rfm.describe()

,CustomerID,Recency,Frequency,Monetary
count,4338.000000,4338.000000,4338.000000,4338.000000
mean,15300.408022,92.536422,4.272015,2054.266460
std,1721.808492,100.014169,7.697998,8989.230441
min,12346.000000,1.000000,1.000000,3.750000
25%,13813.250000,18.000000,1.000000,307.415000
50%,15299.500000,51.000000,2.000000,674.485000
75%,16778.750000,142.000000,5.000000,1661.740000
max,18287.000000,374.000000,209.000000,280206.020000


### RFM Scoring 
- Method: Quantile-based Scoring (Quintile)

RFM scores (1-5) were assigned using quantile-based segmentation 


> Note: Recency is reversed — lower value = better = higher score

### Output

| Column | Description | Example |
|:-------|:------------|:--------|
| RFM_Score | String (R+F+M) | "545" |
| RFM_Total | Sum (R+F+M) | 14 |

### Issue & Solution

| Attempt | Issue | Solution |
|:--------|:------|:---------|
| `pd.qcut()` | Duplicate bin edges | Added `.rank(method='first')` |

In [983]:
# RFM Scoring
rfm['R'] = pd.qcut(rfm['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])

# create RFM_Segment and RFM_Score columns to combine R, F, M values and sum them
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

rfm.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score
0,12346.0,326,1,77183.60,1,1,5,115,7
1,12347.0,2,7,4310.00,5,5,5,555,15
2,12348.0,75,4,1797.24,2,4,4,244,10
3,12349.0,19,1,1757.55,4,1,4,414,9
4,12350.0,310,1,334.40,1,1,2,112,4


In [984]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4338 entries, 0 to 4337
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   CustomerID   4338 non-null   float64 
 1   Recency      4338 non-null   int64   
 2   Frequency    4338 non-null   int64   
 3   Monetary     4338 non-null   float64 
 4   R            4338 non-null   category
 5   F            4338 non-null   category
 6   M            4338 non-null   category
 7   RFM_Segment  4338 non-null   object  
 8   RFM_Score    4338 non-null   int64   
dtypes: category(3), float64(2), int64(3), object(1)
memory usage: 216.8+ KB


In [985]:
# Define segmentation function for baseline segments
def assign_segment(score):
    if score >= 12:
        return 'High-Value'
    elif score >= 7:
        return 'Mid-Value'
    else:
        return 'Low-Value'

# Apply segmentation
rfm['Segment'] = rfm['RFM_Score'].apply(assign_segment)

# Count customers per segment
print(rfm['Segment'].value_counts())

Segment
Mid-Value     1785
Low-Value     1309
High-Value    1244
Name: count, dtype: int64


### RFM Matrix: Where Are Your Best Customers?
**Purpose:** Visualize average spending across R & F scores — identify which customer behaviors drive the most revenue.

In [986]:
# RFM Heatmap - Average Monetary
rfm_monetary = rfm.groupby(['R', 'F'])['Monetary'].mean().unstack(fill_value=0)

fig = px.imshow(rfm_monetary,
                title='RFM Heatmap (Average Monetary by R & F Score)',
                labels=dict(x='Frequency Score', y='Recency Score', color='Avg Monetary'),
                color_continuous_scale='Viridis',
                text_auto='.0f')
fig.show()

/var/folders/y_/phkrzxtn25z9pssz73ykz4fh0000gn/T/ipykernel_82833/3923713405.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Insight:** 
- VIP customers (R=5, F=5) spend 24x more than new customers (£9,204 vs £388)
- Frequency is the strongest driver of monetary value
- Focus: Convert new customers to repeat buyers

# Insight from RFM Heatmap

1. **VIP Customers (bottom-right - yellow)**
   - R=5, F=5 → £9,204
   - Recent + Frequent = Highest spending
   - Action: Retain with loyalty program

2. **Frequency matters most**
   - Look at right column (F=5): £2,399 → £2,872 → £4,198 → £4,746 → £9,204
   - More frequent → Higher spending

3. **Lost VIP (top-right)**
   - R=1, F=5 → £2,399
   - Used to buy often but inactive now
   - Action: Re-engagement campaign

4. **New Customers (bottom-left)**
   - R=5, F=1 → £388
   - Recent but only 1 purchase
   - Action: Upselling / encourage repeat purchase

### Segment Analysis: Customer Count & Average Spending
**Purpose:** Compare segment size (area) with average monetary value (color) — identify which segments drive revenue.

In [987]:
# Treemap - show segment size & value
segment_summary = rfm.groupby('Segment').agg({
    'CustomerID': 'count',
    'Monetary': 'mean'
}).reset_index()
segment_summary.columns = ['Segment', 'Count', 'Avg_Monetary']

fig = px.treemap(segment_summary,
                 path=['Segment'],
                 values='Count',
                 color='Avg_Monetary',
                 color_continuous_scale='Viridis',
                 title='Customer Segments: Size & Value')
fig.show()

**Insight:**
- **High-Value (yellow):** 28.7% of customers, avg £5,474 — smallest group but highest value
- **Mid-Value (purple):** 41.1% of customers, avg £972 — largest growth opportunity
- **Low-Value (dark):** 30.2% of customers, avg £280 — 20x less than High-Value
- **Key:** High-Value customers spend 20x more than Low-Value despite similar group sizes

### Customer Segment Proportion
**Purpose:** Overview of customer portfolio — understand segment sizes for resource allocation

In [988]:
# Pie chart - segment proportion
segment_count = rfm['Segment'].value_counts()

fig = px.pie(values=segment_count.values,
             names=segment_count.index,
             title='Customer Segment Distribution',
             color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.show()

# Print counts
print(rfm['Segment'].value_counts())

Segment
Mid-Value     1785
Low-Value     1309
High-Value    1244
Name: count, dtype: int64


### Frequency vs Monetary: Identifying High-Spending Customers

In [989]:
# 2D Scatter - Frequency vs Monetary 
fig = px.scatter(rfm,
                 x='Frequency',
                 y='Monetary',
                 color='Segment',
                 title='Frequency vs Monetary by Segment',
                 opacity=0.6)
fig.update_traces(marker=dict(size=5))
fig.show()

**Insight from Frequency vs Monetary:**
- **Low-Value (yellow):** Clustered at bottom-left — low frequency, low spending
- **High-Value (green):** Spread toward top-right — high frequency, high spending
- **Outliers:** Some customers with F=200+, M=£280K — Super VIP
- **Pattern:** Strong positive correlation between Frequency and Monetary

### Recency vs Frequency: Churn Risk Detection

In [990]:
# R vs F Scatter to see how many customers fall into low recency & high frequency 
fig = px.scatter(rfm,
                 x='Recency',
                 y='Frequency',
                 color='Segment',
                 title='Recency vs Frequency by Segment',
                 opacity=0.6)
fig.update_traces(marker=dict(size=5))
fig.show()

## Part 7. Customer Segmentation & Churn Prediction

### 7.1 Data Preparation
- 7.1.1 StandardScaler only

In [991]:
X = rfm[['Recency', 'Frequency', 'Monetary']]

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Data shape: {X_scaled.shape}")
print(f"\nFeature ranges (before scaling):")
print(f"Recency:   {X['Recency'].min():,.0f} - {X['Recency'].max():,.0f} days")
print(f"Frequency: {X['Frequency'].min():,.0f} - {X['Frequency'].max():,.0f} purchases")
print(f"Monetary:  £{X['Monetary'].min():,.0f} - £{X['Monetary'].max():,.0f}")

Data shape: (4338, 3)

Feature ranges (before scaling):
Recency:   1 - 374 days
Frequency: 1 - 209 purchases
Monetary:  £4 - £280,206


**Note:** Features have different scales — normalization required for distance-based clustering (K-Means).

**Selected:** K-Means — significantly higher Silhouette Score (0.5460 vs 0.1515) indicates better cluster separation for this dataset.

### 7.2 : K selection 

In [992]:
# Scale data (for K selection)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Test K from 2 to 10
k_range = range(2, 11)
inertia = []
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Elbow Chart
fig = px.line(x=list(k_range), y=inertia, markers=True,
              title='Elbow Method',
              labels={'x': 'Number of Clusters (K)', 'y': 'Inertia'})
fig.show()

# Silhouette Chart
fig = px.line(x=list(k_range), y=silhouette_scores, markers=True,
              title='Silhouette Score by K',
              labels={'x': 'Number of Clusters (K)', 'y': 'Silhouette Score'})
fig.show()

# Print results
for k, iner, sil in zip(k_range, inertia, silhouette_scores):
    print(f"K={k}: Inertia = {iner:.0f}, Silhouette = {sil:.4f}")

K=2: Inertia = 9013, Silhouette = 0.8958
K=3: Inertia = 5439, Silhouette = 0.5942
K=4: Inertia = 4092, Silhouette = 0.6162
K=5: Inertia = 3118, Silhouette = 0.6165
K=6: Inertia = 2473, Silhouette = 0.5963
K=7: Inertia = 2022, Silhouette = 0.5165
K=8: Inertia = 1716, Silhouette = 0.4859
K=9: Inertia = 1446, Silhouette = 0.4777
K=10: Inertia = 1292, Silhouette = 0.4214


**Observation:** K=4 and K=5 have similar Silhouette Scores. Further business analysis needed to determine optimal K.

7.2.1 K=4 vs K=5 Business Comparison

**Key Difference:**
- K=4: Combines all VIP customers into one group (13 customers, 18.6% revenue)
- K=5: Separates Ultra VIP (6 customers, £190K avg, 12.9% revenue) from Super VIP (8 customers, £55K avg, 5.0% revenue)

**Selected:** K=5 — provides more granular segmentation for targeted marketing strategies

In [993]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("K=4 Revenue Distribution", "K=5 Revenue Distribution"),
                    specs=[[{"type": "pie"}, {"type": "pie"}]])

# K=4 Pie Chart
fig.add_trace(go.Pie(labels=[f'Cluster {i}' for i in k4_profile.index],
                     values=k4_profile['Revenue%'],
                     textinfo='label+percent',
                     hole=0.3), row=1, col=1)

# K=5 Pie Chart
fig.add_trace(go.Pie(labels=[f'Cluster {i}' for i in k5_profile.index],
                     values=k5_profile['Revenue%'],
                     textinfo='label+percent',
                     hole=0.3), row=1, col=2)

fig.update_layout(title_text="Revenue Distribution: K=4 vs K=5", height=400)
fig.show()

### 7.3 Normalization & Algorithm Comparison (K=5)

In [994]:
# Method 1: StandardScaler
scaler_std = StandardScaler()
X_standard = scaler_std.fit_transform(X)

# Method 2: Log + StandardScaler
X_log = X.apply(lambda x: np.log1p(x))
scaler_log = StandardScaler()
X_log_scaled = scaler_log.fit_transform(X_log)

# Compare all combinations
results = []

# StandardScaler + K-Means
labels = KMeans(n_clusters=5, random_state=42, n_init=10).fit_predict(X_standard)
results.append(['StandardScaler', 'K-Means', silhouette_score(X_standard, labels)])

# StandardScaler + GMM
labels = GaussianMixture(n_components=5, random_state=42).fit_predict(X_standard)
results.append(['StandardScaler', 'GMM', silhouette_score(X_standard, labels)])

# Log+StandardScaler + K-Means
labels = KMeans(n_clusters=5, random_state=42, n_init=10).fit_predict(X_log_scaled)
results.append(['Log+StandardScaler', 'K-Means', silhouette_score(X_log_scaled, labels)])

# Log+StandardScaler + GMM
labels = GaussianMixture(n_components=5, random_state=42).fit_predict(X_log_scaled)
results.append(['Log+StandardScaler', 'GMM', silhouette_score(X_log_scaled, labels)])

# Display results
results_df = pd.DataFrame(results, columns=['Normalization', 'Algorithm', 'Silhouette'])
results_df = results_df.sort_values('Silhouette', ascending=False)

print("=" * 55)
print("Normalization & Algorithm Comparison (K=5)")
print("=" * 55)
print(results_df.to_string(index=False))
print("=" * 55)
print(f"✓ Best: {results_df.iloc[0]['Normalization']} + {results_df.iloc[0]['Algorithm']}")

Normalization & Algorithm Comparison (K=5)
     Normalization Algorithm  Silhouette
    StandardScaler   K-Means    0.616523
Log+StandardScaler   K-Means    0.316097
Log+StandardScaler       GMM    0.153019
    StandardScaler       GMM    0.149755
✓ Best: StandardScaler + K-Means


### 7.4 Final Clustering (K=5, StandardScaler, K-Means)
- 7.4.1 Cluster Profile

In [995]:
kmeans_final = KMeans(n_clusters=5, random_state=42, n_init=10)
rfm['Cluster'] = kmeans_final.fit_predict(X_scaled)

# Cluster Profile
cluster_profile = rfm.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'sum', 'count']
}).round(0)
cluster_profile.columns = ['Recency', 'Frequency', 'Monetary', 'Revenue', 'Customers']
cluster_profile['Revenue%'] = (cluster_profile['Revenue'] / cluster_profile['Revenue'].sum() * 100).round(1)

print(cluster_profile)

         Recency  Frequency  Monetary    Revenue  Customers  Revenue%
Cluster                                                              
0           44.0        4.0    1339.0  4082814.0       3049      45.8
1          249.0        2.0     478.0   507750.0       1062       5.7
2           16.0       21.0   12832.0  2733161.0        213      30.7
3            6.0      120.0   55313.0   442501.0          8       5.0
4            8.0       43.0  190863.0  1145181.0          6      12.9


- 7.4.2  Patero analysis
**Purpose:** Identify revenue concentration — how few customers drive most revenue.

In [996]:
# Pareto Analysis
rfm_sorted = rfm.sort_values('Monetary', ascending=False)
rfm_sorted['Cumulative_Revenue'] = rfm_sorted['Monetary'].cumsum()
rfm_sorted['Cumulative_Pct'] = rfm_sorted['Cumulative_Revenue'] / rfm_sorted['Monetary'].sum() * 100

# Top 20% customers
top_20 = int(len(rfm_sorted) * 0.2)
revenue_pct_20 = rfm_sorted.iloc[:top_20]['Monetary'].sum() / rfm_sorted['Monetary'].sum() * 100
print(f"Top 20% customers ({top_20} people) contribute: {revenue_pct_20:.1f}% of revenue")

# Top customers for 80% revenue
top_80 = (rfm_sorted['Cumulative_Pct'] <= 80).sum()
print(f"Top {top_80} customers ({top_80/len(rfm_sorted)*100:.1f}%) contribute to 80% of revenue")


Top 20% customers (867 people) contribute: 74.6% of revenue
Top 1132 customers (26.1%) contribute to 80% of revenue


In [997]:
# Pareto Chart
fig = px.line(x=range(1, len(rfm_sorted)+1), 
              y=rfm_sorted['Cumulative_Pct'].values,
              title='Pareto Chart: Cumulative Revenue by Customer',
              labels={'x': 'Number of Customers', 'y': 'Cumulative Revenue %'})
fig.add_hline(y=80, line_dash="dash", line_color="red", annotation_text="80%")
fig.add_hline(y=100, line_dash="dash", line_color="gray")
fig.show()

print(f"Top 20% customers ({top_20} people) contribute: {revenue_pct_20:.1f}% of revenue")


Top 20% customers (867 people) contribute: 74.6% of revenue


**Insight:**
- Top 20% customers (867) generate 74.6% of revenue — close to Pareto 80/20 rule
- Ultra VIP (6 customers) average £190K each — losing 1 = significant revenue loss

**Business Implication:**
- Prioritize retention for top-tier customers
- At-Risk customers in top 26% = urgent churn prevention

### 7.5 Churn Definition 


In [998]:
# Recency Distribution
fig = px.histogram(rfm, x='Recency', nbins=50,
                   title='Recency Distribution: When Do Customers Stop Returning?',
                   labels={'Recency': 'Days Since Last Purchase', 'count': 'Number of Customers'})
fig.add_vline(x=50, line_dash="dash", line_color="red", annotation_text="50 days (Median)")
fig.show()

print(f"Mean Recency: {rfm['Recency'].mean():.0f} days")
print(f"Median Recency: {rfm['Recency'].median():.0f} days")
print(f"Std Recency: {rfm['Recency'].std():.0f} days")

Mean Recency: 93 days
Median Recency: 51 days
Std Recency: 100 days


**Churn Threshold:** 50 days (Median Recency)

- 50% of customers return within 50 days
- Customers exceeding this threshold = abnormal behavior = potential churn

### 7.6 Time-based Split (Avoid Data Leakage)
**Time-based Split because :**
- Initial attempt using random split caused data leakage
- Future customer behavior leaked into training data
- Time-based split simulates real-world prediction scenario

**Split:**
- Training: Dec 2010 - May 2011 (6 months - stable period) Build RFM features
- Label: Jun - Aug 2011 (3 months - before Sep spike) Define churn

In [999]:
# Time-based Split
# Training: Dec 2010 - May 2011 (6 months - stable period)
# Label: Jun - Aug 2011 (3 months - before Sep spike)


training_start = '2010-12-01'
training_end = '2011-05-31'
label_start = '2011-06-01'
label_end = '2011-08-31'

data_train = data_clean[(data_clean['InvoiceDate'] >= training_start) & 
                         (data_clean['InvoiceDate'] <= training_end)]

data_label = data_clean[(data_clean['InvoiceDate'] >= label_start) & 
                         (data_clean['InvoiceDate'] <= label_end)]

print(f"Training transactions: {len(data_train):,}")
print(f"Label transactions: {len(data_label):,}")
print(f"Training Period: {training_start} to {training_end}")
print(f"Label Period: {label_start} to {label_end}")

Training transactions: 144,541
Label transactions: 80,262
Training Period: 2010-12-01 to 2011-05-31
Label Period: 2011-06-01 to 2011-08-31


### 7.7 RFM Features and Churn Label
- Training Period

In [1000]:
# Calculate RFM from TRAINING data only
training_end = '2011-05-31'  # Updated to match new split
analysis_date = pd.to_datetime(training_end) + pd.Timedelta(days=1)

recency = data_train.groupby('CustomerID')['InvoiceDate'].max()
recency = (analysis_date - recency).dt.days

frequency = data_train.groupby('CustomerID')['InvoiceNo'].nunique()

monetary = data_train.groupby('CustomerID')['TotalAmount'].sum()

rfm_train = pd.DataFrame({
    'Recency': recency,
    'Frequency': frequency,
    'Monetary': monetary
}).reset_index()

print(f"\nCustomers in training: {len(rfm_train):,}")


Customers in training: 2,708


- Churn Label Creation
**Churn Definition:** Customer who purchased in training period but did NOT purchase in label period (Jun-Aug 2011)


In [1001]:
# Create Churn label
customers_in_label = data_label['CustomerID'].unique()
rfm_train['Churn'] = ~rfm_train['CustomerID'].isin(customers_in_label)
rfm_train['Churn'] = rfm_train['Churn'].astype(int)

print(f"\nChurn rate: {rfm_train['Churn'].mean()*100:.1f}%")
print(rfm_train['Churn'].value_counts())


Churn rate: 50.5%
Churn
1    1367
0    1341
Name: count, dtype: int64


| Churn | Count | % |
|-------|-------|---|
| 0 (Active) | 1,936 | 53.7% |
| 1 (Churned) | 1,668 | 46.3% |

**Note:**
- Training: Dec 2010 - May 2011 (stable period)
- Label: Jun - Aug 2011 (before holiday spike)
- 46.3% churn rate reflects normal customer behavior

**Insight:**
- Balanced dataset 
- No need for oversampling/undersampling
- Nearly half of training customers did not return in Jun-Aug 2011

**Limitation:**
- Dataset covers only 13 months (Dec 2010 - Dec 2011)
- Cannot observe yearly seasonality or long-term trends
- Model trained on 6-month window may not generalize to other periods

### 7.8 Churn Prediction Model
- Prepare features + Scale + Split

In [1002]:
# Prepare features and target
X = rfm_train[['Recency', 'Frequency', 'Monetary']]
y = rfm_train['Churn']

# Scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Features shape: {X_scaled.shape}")

Features shape: (2708, 3)


In [1003]:
# Train-test split (80/20)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

Train size: 2166
Test size: 542


### 7.9 Model Comparison
**Models to compare:**
- Logistic Regression 
- Decision Tree 
- Random Forest 
- Gradient Boosting 
### Critiria
- All interpretable with feature importance
- Excluded: SVM, Neural Network (less interpretable), XGBoost (similar to GB)

In [1004]:
# Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("=== Logistic Regression ===")
print(classification_report(y_test, y_pred_lr))

=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.72      0.59      0.65       260
           1       0.68      0.79      0.73       282

    accuracy                           0.70       542
   macro avg       0.70      0.69      0.69       542
weighted avg       0.70      0.70      0.69       542



In [1005]:
# Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print("=== Decision Tree ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_dt):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_dt):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_dt))

=== Decision Tree ===
Accuracy: 0.5535
F1-Score: 0.5724
Recall: 0.5745

Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.53      0.53       260
           1       0.57      0.57      0.57       282

    accuracy                           0.55       542
   macro avg       0.55      0.55      0.55       542
weighted avg       0.55      0.55      0.55       542



In [1006]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("=== Random Forest ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

=== Random Forest ===
Accuracy: 0.6255
F1-Score: 0.6530
Recall: 0.6773

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.57      0.59       260
           1       0.63      0.68      0.65       282

    accuracy                           0.63       542
   macro avg       0.62      0.62      0.62       542
weighted avg       0.63      0.63      0.62       542



In [1007]:
# Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

print("=== Gradient Boosting ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_gb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_gb):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb))

=== Gradient Boosting ===
Accuracy: 0.6956
F1-Score: 0.7291
Recall: 0.7872

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.60      0.65       260
           1       0.68      0.79      0.73       282

    accuracy                           0.70       542
   macro avg       0.70      0.69      0.69       542
weighted avg       0.70      0.70      0.69       542



**Top 2 Candidates:**
- Logistic Regression (Recall: 0.79, F1: 0.73)
- Gradient Boosting (Recall: 0.79, F1: 0.73)

**Next Step:** Cross Validation to compare stability and overfitting risk

### 7.10 Cross Validation

**Purpose:** Validate model performance is consistent

In [1016]:
# Cross-Validation Comparison
print("\n=== Cross-Validation Comparison ===")
lr = LogisticRegression(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

# Logistic Regression
cv_accuracy_lr = cross_val_score(lr, X_scaled, y, cv=5, scoring='accuracy')
cv_f1_lr = cross_val_score(lr, X_scaled, y, cv=5, scoring='f1')
cv_recall_lr = cross_val_score(lr, X_scaled, y, cv=5, scoring='recall')

print("=== Logistic Regression ===")
print(f"Accuracy: {cv_accuracy_lr.mean():.4f} (+/- {cv_accuracy_lr.std():.4f})")
print(f"F1-Score: {cv_f1_lr.mean():.4f} (+/- {cv_f1_lr.std():.4f})")
print(f"Recall: {cv_recall_lr.mean():.4f} (+/- {cv_recall_lr.std():.4f})")

# Gradient Boosting
cv_accuracy_gb = cross_val_score(gb, X_scaled, y, cv=5, scoring='accuracy')
cv_f1_gb = cross_val_score(gb, X_scaled, y, cv=5, scoring='f1')
cv_recall_gb = cross_val_score(gb, X_scaled, y, cv=5, scoring='recall')

print("\n=== Gradient Boosting ===")
print(f"Accuracy: {cv_accuracy_gb.mean():.4f} (+/- {cv_accuracy_gb.std():.4f})")
print(f"F1-Score: {cv_f1_gb.mean():.4f} (+/- {cv_f1_gb.std():.4f})")
print(f"Recall: {cv_recall_gb.mean():.4f} (+/- {cv_recall_gb.std():.4f})")


=== Cross-Validation Comparison ===
=== Logistic Regression ===
Accuracy: 0.6824 (+/- 0.0179)
F1-Score: 0.7123 (+/- 0.0119)
Recall: 0.7783 (+/- 0.0143)

=== Gradient Boosting ===
Accuracy: 0.6894 (+/- 0.0176)
F1-Score: 0.7098 (+/- 0.0124)
Recall: 0.7520 (+/- 0.0169)


**Selected: Logistic Regression**
- Higher Recall (0.778 > 0.752) — catches more churners
- Lower variance — more stable across folds
- Interpretable — can explain coefficients to business

### 7.11 Confusion Matrix

**Purpose:** Visualize model predictions vs actual results — see where model makes mistakes.

| Term | Meaning |
|------|---------|
| T (True) | Predicted Correct ✅ |
| F (False) | Predicted Wrong ❌ |
| P (Positive) | Predicted as Churn (1) |
| N (Negative) | Predicted as Active (0) |

In [1019]:
# Confusion Matrix for Logistic Regression
cm = confusion_matrix(y_test, y_pred_lr)

# Create labels
labels = [['TN<br>True Negative<br>' + str(cm[0,0]), 
           'FP<br>False Positive<br>' + str(cm[0,1])],
          ['FN<br>False Negative<br>' + str(cm[1,0]), 
           'TP<br>True Positive<br>' + str(cm[1,1])]]

fig = px.imshow(cm, 
                labels=dict(x="Predicted", y="Actual"),
                x=['Active (0)', 'Churn (1)'],
                y=['Active (0)', 'Churn (1)'],
                title='Confusion Matrix - Logistic Regression',
                color_continuous_scale='Plasma')

# Add text annotations
fig.update_traces(text=labels, texttemplate="%{text}", textfont_size=14)
fig.show()

| Term | Count | Meaning |
|------|-------|---------|
| TN (True Negative) | 154 | Predicted Active → Actually Active ✅ |
| TP (True Positive) | 223 | Predicted Churn → Actually Churn ✅ |
| FP (False Positive) | 106 | Predicted Churn → Actually Active ❌ |
| FN (False Negative) | 59 | Predicted Active → Actually Churn ❌ (Dangerous!) |

**Note:** FN = 59 churners we missed — priority to minimize this.

### 7.12 ROC Curve

**Purpose:** Evaluate model's ability to distinguish between Churn and Active customers.

**How to Read:**
- Closer to top-left corner = better model
- AUC (Area Under Curve): 0.5 = random guess, 1.0 = perfect

In [1021]:
# Re-fit Logistic Regression for ROC
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# Get prediction probabilities
y_prob = lr.predict_proba(X_test)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot
fig = px.area(x=fpr, y=tpr,
              title=f'ROC Curve - Logistic Regression (AUC = {roc_auc:.3f})',
              labels=dict(x='False Positive Rate', y='True Positive Rate'))
fig.add_shape(type='line', x0=0, x1=1, y0=0, y1=1, 
              line=dict(dash='dash', color='red'))
fig.show()

print(f"AUC Score: {roc_auc:.3f}")

AUC Score: 0.741


**Insight:**
- AUC 0.741 indicates good discrimination between Churn and Active customers
- Curve is clearly above the red dashed line (random guess = 0.5)
- Model has reasonable predictive power for churn classification

### 7.13 Feature Importance

**Purpose:** Identify which RFM feature contributes most to churn prediction — guide business actions.

In [1028]:
# Feature Importance from Logistic Regression (coefficients)
feature_names = ['Recency', 'Frequency', 'Monetary']
coefficients = lr.coef_[0]

# Create DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': abs(coefficients)
}).sort_values('Abs_Coefficient', ascending=False)


In [1029]:
# Feature Importance 
fig = px.bar(importance_df, 
             x='Coefficient', 
             y='Feature',
             orientation='h',
             title='Feature Importance - Logistic Regression',
             labels={'Coefficient': 'Coefficient (+ = More Churn, - = Less Churn)'},
             color='Coefficient',
             color_continuous_scale='RdBu_r')
fig.add_vline(x=0, line_dash="dash", line_color="black")
fig.show()

| Rank | Feature | Coefficient | Meaning |
|------|---------|-------------|---------|
| 1st | Frequency | -1.67 | Higher frequency → Much less churn |
| 2nd | Recency | +0.30 | Higher recency → More churn |
| 3rd | Monetary | -0.12 | Higher spending → Slightly less churn (weak effect) |

**Insight:**
- **Frequency is the strongest predictor** — customers who buy often are unlikely to churn
- **Recency matters** — inactive customers (high recency) are at risk
- **Monetary has minimal impact** — high spending alone doesn't guarantee retention

In [1025]:
# Create models folder (outside notebooks/)
os.makedirs('../models', exist_ok=True)

# Save model and scaler
joblib.dump(lr, '../models/churn_model.pkl')
joblib.dump(scaler, '../models/scaler.pkl')

print("Model saved to models/ folder")

Model saved to models/ folder


## Part 8. Model Deployment

### 8.1 Churn Prediction Demo (Gradio)

**Purpose:** Interactive demo for predicting customer churn using trained model.

**Features:**
| Tab | Function | Use Case |
|-----|----------|----------|
| Single Customer | Input R, F, M manually | Sales team checking individual customer |
| Batch Upload | Upload CSV file | Marketing team predicting entire customer list |

**Live Demo:** [Hugging Face Space] [https://huggingface.co/spaces/ItsmePannita/churn-prediction]

In [1026]:
# Load model and scaler
model = joblib.load('../models/churn_model.pkl')
scaler = joblib.load('../models/scaler.pkl')

# Single prediction
def predict_single(recency, frequency, monetary):
    features = np.array([[recency, frequency, monetary]])
    features_scaled = scaler.transform(features)
    
    prediction = model.predict(features_scaled)[0]
    probability = model.predict_proba(features_scaled)[0]
    
    if prediction == 1:
        return f"⚠️ CHURN RISK: {probability[1]*100:.1f}%"
    else:
        return f"✅ ACTIVE: {probability[0]*100:.1f}% likely to stay"

# Batch prediction (upload CSV)
def predict_batch(file):
    df = pd.read_csv(file.name)
    
    # Make column names lowercase
    df.columns = df.columns.str.lower().str.strip()
    
    # Map common column names
    column_mapping = {
        'r': 'recency', 'days': 'recency',
        'f': 'frequency', 'purchases': 'frequency',
        'm': 'monetary', 'spending': 'monetary', 'revenue': 'monetary'
    }
    df = df.rename(columns=column_mapping)
    
    # Check required columns
    required = ['recency', 'frequency', 'monetary']
    if not all(col in df.columns for col in required):
        return pd.DataFrame({'Error': ['CSV must have columns: Recency, Frequency, Monetary']})
    
    features = df[required]
    features_scaled = scaler.transform(features)
    
    df['Churn_Prediction'] = model.predict(features_scaled)
    df['Churn_Probability'] = model.predict_proba(features_scaled)[:, 1].round(3)
    
    return df

# Create tabs
with gr.Blocks() as demo:
    gr.Markdown("# Customer Churn Prediction")
    gr.Markdown("Predict customer churn using RFM values")
    
    with gr.Tab("Single Customer"):
        recency = gr.Number(label="Recency (days since last purchase)", value=50)
        frequency = gr.Number(label="Frequency (number of purchases)", value=5)
        monetary = gr.Number(label="Monetary (total spending £)", value=1000)
        output_single = gr.Text(label="Prediction")
        btn_single = gr.Button("Predict")
        btn_single.click(predict_single, [recency, frequency, monetary], output_single)
    
    with gr.Tab("Batch Upload (CSV)"):
        gr.Markdown("**CSV columns:** Recency, Frequency, Monetary (or R, F, M)")
        file_input = gr.File(label="Upload CSV")
        output_batch = gr.Dataframe(label="Results")
        btn_batch = gr.Button("Predict All")
        btn_batch.click(predict_batch, file_input, output_batch)

In [1027]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


## Part 9. Conclusion

### Key Findings

- **Frequency is the strongest churn predictor** (coefficient: -1.67) — customers who buy often don't churn
- Top 20% customers generate 74.6% revenue (Pareto principle)
- Model Performance: Recall 78%, AUC 0.741
- Churn Rate: 46.3% (based on stable period Jun-Aug 2011)

### Recommendations

- Focus retention on high-frequency customers, not just high spenders
- Re-engage customers inactive >50 days before they churn
- Prioritize Ultra VIP & At-Risk segments for personalized campaigns
- Increase AOV (currently £303 median) with bundle deals above £400

### Limitations

- Data: 13 months only (Dec 2010 - Dec 2011), UK-dominant market (92.6%)
- Model: Trained on 6-month window (Dec 2010 - May 2011), may not generalize to holiday periods
- No customer demographics or product category features